In [3]:
import pandas as pd
from calendar import monthrange
from math import ceil
import numpy as np
import datetime as dt

data = pd.read_csv('./df_input.csv', sep=';', delimiter=';')
data_model = data.copy()

#usando apenas as top 100 linhas (linhas com mais exemplos)
top100_linhalist = data_model.linha.value_counts().index[:100]

In [4]:
## Inserir zero rows
def week_of_month(dt):
    """ Returns the week of the month for the specified date.
    """

    first_day = dt.replace(day=1)

    dom = dt.day
    adjusted_dom = dom + first_day.weekday()

    return int(ceil(adjusted_dom/7.0))

# linha_list = data_model.linha.unique()


for linha in top100_linhalist:
    currentDataModel_Linha = data_model.loc[data_model.linha == linha]
    mes_max = currentDataModel_Linha.mes.max()
    mes_min = currentDataModel_Linha.mes.min()
    for mes in range(mes_min, mes_max+1):
        currentDataModel_Mes = currentDataModel_Linha.loc[currentDataModel_Linha.mes == mes]
        dia_max = monthrange(2020, mes)[1]
        for dia in range(1, dia_max+1):
            currentDataModel_Dia = currentDataModel_Mes.loc[currentDataModel_Mes.d_mes == dia]
            
            for hora in range(0, 24):
                currentDataModel_Hora = currentDataModel_Dia[currentDataModel_Dia.hora == hora]
                if currentDataModel_Hora.empty == False:
                    continue
                else:
                    # print('INSERINDO NO MES '+str(mes)+' NO DIA '+str(dia)+ ' NA HORA '+str(hora))
                    dd = dt.datetime.strptime('2020-'+str(mes)+'-'+str(dia)+' '+str(hora)+':00:00', "%Y-%m-%d %H:00:00")
                    new_row = {
                            'linha': linha,
                            'data_hora': dd,
                            'validations_per_hour': 0,
                            'd_semana': dd.weekday(),
                            'hour_sin': np.sin(2 * np.pi * hora/23.0),
                            'hour_cos': np.cos(2 * np.pi * hora/23.0),
                            'hora': hora,
                            'd_mes': dia,
                            'd_ano': dd.timetuple().tm_yday,
                            'mes': mes,
                            'semana_do_mes': week_of_month(dd)
                        }
                    # print(new_row)
                    data_model = data_model.append(new_row, ignore_index=True)

                #insert validation zero
data_model.loc[data_model.mes == 3]

In [11]:
data_model = data_model.sort_values(['linha', 'data_hora'], ascending=[True, True])

In [16]:
data_model.to_csv('./data_input_zerofill.csv', index=False, sep=';')

In [14]:
data_model

,linha,data_hora,validations_per_hour,d_semana,hour_sin,hour_cos,hora,d_mes,d_ano,mes,semana_do_mes
1421917,1,2020-03-01 00:00:00,0,6,0.000000,1.000000,0,1,61,3,1
1421918,1,2020-03-01 01:00:00,0,6,0.269797,0.962917,1,1,61,3,1
1421919,1,2020-03-01 02:00:00,0,6,0.519584,0.854419,2,1,61,3,1
1421920,1,2020-03-01 03:00:00,0,6,0.730836,0.682553,3,1,61,3,1
1421921,1,2020-03-01 04:00:00,0,6,0.887885,0.460065,4,1,61,3,1
...,...,...,...,...,...,...,...,...,...,...,...
1278857,999,2020-12-01 08:00:00,12,1,0.816970,-0.576680,8,1,336,12,0
1278858,999,2020-12-01 09:00:00,1,1,0.631088,-0.775711,9,1,336,12,0
1278859,999,2020-12-01 10:00:00,1,1,0.398401,-0.917211,10,1,336,12,0
1278860,999,2020-12-01 11:00:00,5,1,0.136167,-0.990686,11,1,336,12,0
